In [1]:
from mia_models import train_target_model, load_shadow_data, train_shadow_models
from mia_models import prepare_attack_test_data, prep_validation_data, prep_attack_train_data
from mia_models import shokri_attack, prety_print_result
import tensorflow.keras as keras
import numpy as np
from sklearn.utils import resample
import pandas as pd
import pickle
import os
import csv
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.linear_model import LogisticRegression
from statistics import mean
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from keras.regularizers import l2
from sklearn.metrics import accuracy_score
MODEL_PATH = './model/'
DATA_PATH = './data/'

2023-06-06 11:48:37.271711: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-06 11:48:37.271878: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
# --------------------------------------------Original Data--------------------------------------------------------------#

In [9]:
# original dataset with random index
dataset_name = 'adult'
train_size = 100
attack_test_size = 50
test_start = 100
data = pd.read_csv('data/Adult/adult_original_mod.csv', na_values=["?"], header=None)
data.dropna(inplace=True)
target_dataset = data.sample(n = 150, replace = False)
df_rest = data.loc[~data.index.isin(target_dataset.index)]
shadow_dataset = df_rest.sample(n = 2000, replace = False)
df_rest = df_rest.loc[~df_rest.index.isin(shadow_dataset.index)]
attack_test_nonmembers = df_rest.sample(n = attack_test_size, replace = False)
attack_test_members =  target_dataset.iloc[:train_size,:].sample(n = attack_test_size, replace = False)

In [4]:
# trian target model
per_class_sample=5000
channel=0   
epochs=200
n_class = 2
is_synthetic = True
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, n_class, train_size, test_start, is_synthetic)

2023-06-06 11:53:38.235382: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-06 11:53:38.235802: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-06 11:53:38.235908: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-BP5MTC3R): /proc/driver/nvidia/version does not exist
2023-06-06 11:53:38.238559: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



 Target Train acc :  100.0 Target Test acc :  72.00000286102295


In [ ]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 2000

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

In [11]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list, mpred, nmpred = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

2/2 [==============================] - 0s 5ms/step
0
2/2 [==============================] - 0s 6ms/step
1
1/1 [==============================] - 0s 37ms/step
TP: 0     FP: 40     FN: 50     TN: 10
PPV: 0.0000
Advantage: -0.8000
Accuracy:  0.1 Precision:  0.0


In [ ]:
# --------------------------------------------Synthetic Data------------------------------------------------------------#

In [10]:
# synthetic dataset
train_size = 100
attack_test_size = 50
test_start = 100
target_dataset = pd.read_csv('data/Adult/adult_sds_cac_200.csv', na_values=["?"], header=None).iloc[:150]
shadow_dataset = pd.read_csv('data/Adult/adultODS10K_to_25K.csv', na_values=["?"], header=None)
attack_test_nonmembers = pd.read_csv('data/Adult/adultODS25K_to_30K.csv', na_values=["?"], header=None)
attack_test_nonmembers = attack_test_nonmembers.sample(n=attack_test_size, replace=False)
attack_test_members = pd.read_csv('data/Adult/adultODS1_to_5K.csv', na_values=["?"], header=None)
attack_test_members = attack_test_members.iloc[:100,].sample(n=attack_test_size, replace=False)

In [7]:
# trian target model
per_class_sample=5000
channel=0   
epochs=200
n_class = 2
is_synthetic = True
VERBOSE = 0
test_ratio = 0.3

target_model, dim = train_target_model(target_dataset, dataset_name, per_class_sample, epochs, n_class, train_size, test_start, is_synthetic)


 Target Train acc :  98.00000190734863 Target Test acc :  77.99999713897705


In [ ]:
#train shadow model
n_shadow_models = 20
shadow_data_size = 2000

load_shadow_data(shadow_dataset, dataset_name, n_shadow_models, shadow_data_size, test_ratio, is_synthetic)
n_shadow_train_performance, n_shadow_test_performance, n_attack_data, x_shadow_train, y_shadow_train, x_shadow_test, y_shadow_test, shadow_model_init, shadow_accuracy = train_shadow_models(dataset_name, n_shadow_models, n_class, dim, epochs, channel)

In [19]:
#train attack model
attack_test_data = prepare_attack_test_data(dataset_name, attack_test_members, attack_test_nonmembers, target_model, is_synthetic)
mem_validation, nmem_validation = prep_validation_data(attack_test_data)
attack_train_df = prep_attack_train_data(n_attack_data)
pred_membership, ori_membership, TP_idx_list, TN_idx_list, mpred_syn, nmpred_syn = shokri_attack(attack_train_df, mem_validation, nmem_validation, epochs)
tp, fp, fn, tn, precision, advj, acc, recall = prety_print_result (ori_membership,pred_membership)
print('Accuracy: ', acc, 'Precision: ', precision)

2/2 [==============================] - 0s 5ms/step
0
2/2 [==============================] - 0s 3ms/step
1
1/1 [==============================] - 0s 22ms/step
TP: 38     FP: 35     FN: 12     TN: 15
PPV: 0.5205
Advantage: 0.0600
Accuracy:  0.53 Precision:  0.5205479452054794


In [ ]:
# --------------------------------------------Synthetic Data-------------------------------------------------------------#